In [35]:
import sqlalchemy
import pandas as pd
import numpy as np

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct, and_, inspect

In [90]:
engine = create_engine("sqlite:///Data/Earthquakes.db")
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
Base.classes.keys()

['quakes']

In [91]:
sqlPOR = '''select ClosestCity, 
50/cast(max(sum(case when (category = 2) then 1 else 0 END),0.02) as float) as Mag20,  
50/cast(max(sum(case when (category = 2.5) then 1 else 0 END),0.02) as float) as Mag25,  
50/cast(max(sum(case when (category = 3.0) then 1 else 0 END),0.02) as float) as Mag30,  
50/cast(max(sum(case when (category = 3.5) then 1 else 0 END),0.02) as float) as Mag35,  
50/cast(max(sum(case when (category = 4.0) then 1 else 0 END),0.02) as float) as Mag40,
50/cast(max(sum(case when (category = 4.5) then 1 else 0 END),0.02) as float) as Mag45, 
50/cast(max(sum(case when (category = 5.0) then 1 else 0 END),0.02) as float) as Mag50, 
50/cast(max(sum(case when (category = 5.5) then 1 else 0 END),0.02) as float) as Mag55,
50/cast(max(sum(case when (category = 6.0) then 1 else 0 END),0.02) as float) as Mag60, 
50/cast(max(sum(case when (category = 6.5) then 1 else 0 END),0.02) as float) as Mag65, 
50/cast(max(sum(case when (category = 7.0) then 1 else 0 END),0.02) as float) as Mag70,
50/cast(max(sum(case when (category = 7.5) then 1 else 0 END),0.02) as float) as Mag75,
50/cast(max(sum(case when (category = 8.0) then 1 else 0 END),0.02) as float) as Mag80,      
50/cast(max(sum(case when (category = 8.5) then 1 else 0 END),0.02) as float) as Mag85,  
50/cast(max(sum(case when (category = 9.0) then 1 else 0 END),0.02) as float) as Mag90  
from quakes
group by ClosestCity'''

sqlMain = '''select time, latitude, longitude, depth, mag, ClosestCity, category, lastMonth from quakes'''


In [92]:
conn = engine.connect()
POR_df = pd.read_sql(sqlPOR, conn)
POR_df.set_index('ClosestCity',inplace = True)
POR_df

,Mag20,Mag25,Mag30,Mag35,Mag40,Mag45,Mag50,Mag55,Mag60,Mag65,Mag70,Mag75,Mag80,Mag85,Mag90
ClosestCity,,,,,,,,,,,,,,,
Anaheim,2500.0,0.549451,2.173913,5.000000,16.666667,50.000000,50.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Antelope Peak NV,2500.0,2.083333,3.333333,3.571429,16.666667,16.666667,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Avenal CA,2500.0,0.027579,0.067204,0.196078,0.581395,2.272727,7.142857,16.666667,2500.0,50.0,2500.0,2500.0,2500.0,2500.0,2500.0
Azusa,2500.0,0.649351,3.125000,7.142857,25.000000,2500.000000,2500.000000,50.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Bakersfield,2500.0,0.085911,0.314465,1.250000,2.941176,7.142857,16.666667,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Walnut Creek,2500.0,0.213675,0.641026,2.500000,5.555556,2500.000000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
Warm Springs NV,2500.0,0.500000,1.351351,3.846154,10.000000,2500.000000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0
"Willits, CA",2500.0,0.109409,0.304878,1.666667,3.333333,12.500000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0,2500.0


In [93]:
POR_df.loc['Twenty Nine Palms','Mag25']

0.01447596988998263

In [94]:
conn = engine.connect()
Main_df = pd.read_sql(sqlMain, conn)
Main_df

,time,latitude,longitude,depth,mag,ClosestCity,category,lastMonth
0,1975-12-31 02:53:00.00,34.279000,-116.337000,2.330,2.70,Twenty Nine Palms,2.5,1975-12-01 02:53:00.00
1,1975-12-30 23:37:00.00,36.905667,-121.637167,-0.155,2.67,Salinas,2.5,1975-11-30 23:37:00.00
2,1975-12-30 23:04:00.00,36.838667,-121.569833,4.774,2.67,Salinas,2.5,1975-11-30 23:04:00.00
3,1975-12-30 18:10:00.00,35.781000,-121.421000,11.895,3.15,King City NV,3.0,1975-11-30 18:10:00.00
4,1975-12-29 15:07:00.00,36.816333,-121.138833,6.478,3.50,Salinas,3.5,1975-11-29 15:07:00.00
...,...,...,...,...,...,...,...,...
80381,2016-01-03 09:12:00.00,41.848300,-119.601600,6.700,2.50,Honey Lake CA,2.5,2015-12-04 09:12:00.00
80382,2016-01-03 04:35:00.00,41.869000,-119.613700,9.900,2.50,Honey Lake CA,2.5,2015-12-04 04:35:00.00
80383,2016-01-02 06:38:00.00,38.085667,-121.916000,20.430,2.50,Walnut Creek,2.5,2015-12-03 06:38:00.00
80384,2016-01-01 20:02:00.00,40.175500,-122.228833,32.750,2.56,Redding,2.5,2015-12-02 20:02:00.00


In [95]:
periods = np.array([])
for index, row in Main_df.iterrows():
    catagory = f"Mag{int(row['category']*10)}"
    city = row['ClosestCity']
    
    periods = np.append(periods, POR_df.loc[city,catagory])
periods

    

array([0.01447597, 0.02014504, 0.02014504, ..., 0.21367521, 0.17667845,
       0.01844338])

In [96]:
Main_df['POR'] = periods
Main_df

,time,latitude,longitude,depth,mag,ClosestCity,category,lastMonth,POR
0,1975-12-31 02:53:00.00,34.279000,-116.337000,2.330,2.70,Twenty Nine Palms,2.5,1975-12-01 02:53:00.00,0.014476
1,1975-12-30 23:37:00.00,36.905667,-121.637167,-0.155,2.67,Salinas,2.5,1975-11-30 23:37:00.00,0.020145
2,1975-12-30 23:04:00.00,36.838667,-121.569833,4.774,2.67,Salinas,2.5,1975-11-30 23:04:00.00,0.020145
3,1975-12-30 18:10:00.00,35.781000,-121.421000,11.895,3.15,King City NV,3.0,1975-11-30 18:10:00.00,0.060606
4,1975-12-29 15:07:00.00,36.816333,-121.138833,6.478,3.50,Salinas,3.5,1975-11-29 15:07:00.00,0.171233
...,...,...,...,...,...,...,...,...,...
80381,2016-01-03 09:12:00.00,41.848300,-119.601600,6.700,2.50,Honey Lake CA,2.5,2015-12-04 09:12:00.00,0.072780
80382,2016-01-03 04:35:00.00,41.869000,-119.613700,9.900,2.50,Honey Lake CA,2.5,2015-12-04 04:35:00.00,0.072780
80383,2016-01-02 06:38:00.00,38.085667,-121.916000,20.430,2.50,Walnut Creek,2.5,2015-12-03 06:38:00.00,0.213675
80384,2016-01-01 20:02:00.00,40.175500,-122.228833,32.750,2.56,Redding,2.5,2015-12-02 20:02:00.00,0.176678


In [102]:
def CreateSQL(StartDate, EndDate, City):

    sql =  f'''select  
    cast(sum(case when (category = 2) then 1 else 0 END) as float) as Mag20,  
    cast(sum(case when (category = 2.5) then 1 else 0 END) as float) as Mag25,  
    cast(sum(case when (category = 3.0) then 1 else 0 END) as float) as Mag30,  
    cast(sum(case when (category = 3.5) then 1 else 0 END) as float) as Mag35,  
    cast(sum(case when (category = 4.0) then 1 else 0 END) as float) as Mag40,
    cast(sum(case when (category = 4.5) then 1 else 0 END) as float) as Mag45, 
    cast(sum(case when (category = 5.0) then 1 else 0 END) as float) as Mag50, 
    cast(sum(case when (category = 5.5) then 1 else 0 END) as float) as Mag55,
    cast(sum(case when (category = 6.0) then 1 else 0 END) as float) as Mag60, 
    cast(sum(case when (category = 6.5) then 1 else 0 END) as float) as Mag65, 
    cast(sum(case when (category = 7.0) then 1 else 0 END) as float) as Mag70,
    cast(sum(case when (category = 7.5) then 1 else 0 END) as float) as Mag75,
    cast(sum(case when (category = 8.0) then 1 else 0 END) as float) as Mag80,      
    cast(sum(case when (category = 8.5) then 1 else 0 END) as float) as Mag85,  
    cast(sum(case when (category = 9.0) then 1 else 0 END) as float) as Mag90 

    from quakes

    where "{StartDate}" < time AND time < "{EndDate}" And ClosestCity = "{City}"
    '''

    return sql

periods = np.array([])
df = pd.read_sql(CreateSQL('1/1/2016', '1/1/2017', 'xxx'), conn)
df.dropna(inplace=True)

for index, row in Main_df.iterrows():
    
    if index /10 == int(index /10):
        print(f'{index} rows processed')
        
    
    catagory = f"Mag{int(row['category']*10)}"
    city = row['ClosestCity']
    EndDate = row['time']
    StartDate = row['lastMonth']
    
    
    periods = np.append(periods, POR_df.loc[city,catagory])
    df = df.append(pd.read_sql(CreateSQL(StartDate, EndDate, city),conn))
    
df



0 rows processed
10 rows processed
20 rows processed
30 rows processed
40 rows processed
50 rows processed
60 rows processed
70 rows processed
80 rows processed
90 rows processed
100 rows processed
110 rows processed
120 rows processed
130 rows processed
140 rows processed
150 rows processed
160 rows processed
170 rows processed
180 rows processed
190 rows processed
200 rows processed
210 rows processed
220 rows processed
230 rows processed
240 rows processed
250 rows processed
260 rows processed
270 rows processed
280 rows processed
290 rows processed
300 rows processed
310 rows processed
320 rows processed
330 rows processed
340 rows processed
350 rows processed
360 rows processed
370 rows processed
380 rows processed
390 rows processed
400 rows processed
410 rows processed
420 rows processed
430 rows processed
440 rows processed
450 rows processed
460 rows processed
470 rows processed
480 rows processed
490 rows processed
500 rows processed
510 rows processed
520 rows processed
530 

4160 rows processed
4170 rows processed
4180 rows processed
4190 rows processed
4200 rows processed
4210 rows processed
4220 rows processed
4230 rows processed
4240 rows processed
4250 rows processed
4260 rows processed
4270 rows processed
4280 rows processed
4290 rows processed
4300 rows processed
4310 rows processed
4320 rows processed
4330 rows processed
4340 rows processed
4350 rows processed
4360 rows processed
4370 rows processed
4380 rows processed
4390 rows processed
4400 rows processed
4410 rows processed
4420 rows processed
4430 rows processed
4440 rows processed
4450 rows processed
4460 rows processed
4470 rows processed
4480 rows processed
4490 rows processed
4500 rows processed
4510 rows processed
4520 rows processed
4530 rows processed
4540 rows processed
4550 rows processed
4560 rows processed
4570 rows processed
4580 rows processed
4590 rows processed
4600 rows processed
4610 rows processed
4620 rows processed
4630 rows processed
4640 rows processed
4650 rows processed


8260 rows processed
8270 rows processed
8280 rows processed
8290 rows processed
8300 rows processed
8310 rows processed
8320 rows processed
8330 rows processed
8340 rows processed
8350 rows processed
8360 rows processed
8370 rows processed
8380 rows processed
8390 rows processed
8400 rows processed
8410 rows processed
8420 rows processed
8430 rows processed
8440 rows processed
8450 rows processed
8460 rows processed
8470 rows processed
8480 rows processed
8490 rows processed
8500 rows processed
8510 rows processed
8520 rows processed
8530 rows processed
8540 rows processed
8550 rows processed
8560 rows processed
8570 rows processed
8580 rows processed
8590 rows processed
8600 rows processed
8610 rows processed
8620 rows processed
8630 rows processed
8640 rows processed
8650 rows processed
8660 rows processed
8670 rows processed
8680 rows processed
8690 rows processed
8700 rows processed
8710 rows processed
8720 rows processed
8730 rows processed
8740 rows processed
8750 rows processed


12250 rows processed
12260 rows processed
12270 rows processed
12280 rows processed
12290 rows processed
12300 rows processed
12310 rows processed
12320 rows processed
12330 rows processed
12340 rows processed
12350 rows processed
12360 rows processed
12370 rows processed
12380 rows processed
12390 rows processed
12400 rows processed
12410 rows processed
12420 rows processed
12430 rows processed
12440 rows processed
12450 rows processed
12460 rows processed
12470 rows processed
12480 rows processed
12490 rows processed
12500 rows processed
12510 rows processed
12520 rows processed
12530 rows processed
12540 rows processed
12550 rows processed
12560 rows processed
12570 rows processed
12580 rows processed
12590 rows processed
12600 rows processed
12610 rows processed
12620 rows processed
12630 rows processed
12640 rows processed
12650 rows processed
12660 rows processed
12670 rows processed
12680 rows processed
12690 rows processed
12700 rows processed
12710 rows processed
12720 rows pr

16160 rows processed
16170 rows processed
16180 rows processed
16190 rows processed
16200 rows processed
16210 rows processed
16220 rows processed
16230 rows processed
16240 rows processed
16250 rows processed
16260 rows processed
16270 rows processed
16280 rows processed
16290 rows processed
16300 rows processed
16310 rows processed
16320 rows processed
16330 rows processed
16340 rows processed
16350 rows processed
16360 rows processed
16370 rows processed
16380 rows processed
16390 rows processed
16400 rows processed
16410 rows processed
16420 rows processed
16430 rows processed
16440 rows processed
16450 rows processed
16460 rows processed
16470 rows processed
16480 rows processed
16490 rows processed
16500 rows processed
16510 rows processed
16520 rows processed
16530 rows processed
16540 rows processed
16550 rows processed
16560 rows processed
16570 rows processed
16580 rows processed
16590 rows processed
16600 rows processed
16610 rows processed
16620 rows processed
16630 rows pr

...

66990 rows processed
67000 rows processed
67010 rows processed
67020 rows processed
67030 rows processed
67040 rows processed
67050 rows processed
67060 rows processed
67070 rows processed
67080 rows processed
67090 rows processed
67100 rows processed
67110 rows processed
67120 rows processed
67130 rows processed
67140 rows processed
67150 rows processed
67160 rows processed
67170 rows processed
67180 rows processed
67190 rows processed
67200 rows processed
67210 rows processed
67220 rows processed
67230 rows processed
67240 rows processed
67250 rows processed
67260 rows processed
67270 rows processed
67280 rows processed
67290 rows processed
67300 rows processed
67310 rows processed
67320 rows processed
67330 rows processed
67340 rows processed
67350 rows processed
67360 rows processed
67370 rows processed
67380 rows processed
67390 rows processed
67400 rows processed
67410 rows processed
67420 rows processed
67430 rows processed
67440 rows processed
67450 rows processed
67460 rows pr

70900 rows processed
70910 rows processed
70920 rows processed
70930 rows processed
70940 rows processed
70950 rows processed
70960 rows processed
70970 rows processed
70980 rows processed
70990 rows processed
71000 rows processed
71010 rows processed
71020 rows processed
71030 rows processed
71040 rows processed
71050 rows processed
71060 rows processed
71070 rows processed
71080 rows processed
71090 rows processed
71100 rows processed
71110 rows processed
71120 rows processed
71130 rows processed
71140 rows processed
71150 rows processed
71160 rows processed
71170 rows processed
71180 rows processed
71190 rows processed
71200 rows processed
71210 rows processed
71220 rows processed
71230 rows processed
71240 rows processed
71250 rows processed
71260 rows processed
71270 rows processed
71280 rows processed
71290 rows processed
71300 rows processed
71310 rows processed
71320 rows processed
71330 rows processed
71340 rows processed
71350 rows processed
71360 rows processed
71370 rows pr

74810 rows processed
74820 rows processed
74830 rows processed
74840 rows processed
74850 rows processed
74860 rows processed
74870 rows processed
74880 rows processed
74890 rows processed
74900 rows processed
74910 rows processed
74920 rows processed
74930 rows processed
74940 rows processed
74950 rows processed
74960 rows processed
74970 rows processed
74980 rows processed
74990 rows processed
75000 rows processed
75010 rows processed
75020 rows processed
75030 rows processed
75040 rows processed
75050 rows processed
75060 rows processed
75070 rows processed
75080 rows processed
75090 rows processed
75100 rows processed
75110 rows processed
75120 rows processed
75130 rows processed
75140 rows processed
75150 rows processed
75160 rows processed
75170 rows processed
75180 rows processed
75190 rows processed
75200 rows processed
75210 rows processed
75220 rows processed
75230 rows processed
75240 rows processed
75250 rows processed
75260 rows processed
75270 rows processed
75280 rows pr

78720 rows processed
78730 rows processed
78740 rows processed
78750 rows processed
78760 rows processed
78770 rows processed
78780 rows processed
78790 rows processed
78800 rows processed
78810 rows processed
78820 rows processed
78830 rows processed
78840 rows processed
78850 rows processed
78860 rows processed
78870 rows processed
78880 rows processed
78890 rows processed
78900 rows processed
78910 rows processed
78920 rows processed
78930 rows processed
78940 rows processed
78950 rows processed
78960 rows processed
78970 rows processed
78980 rows processed
78990 rows processed
79000 rows processed
79010 rows processed
79020 rows processed
79030 rows processed
79040 rows processed
79050 rows processed
79060 rows processed
79070 rows processed
79080 rows processed
79090 rows processed
79100 rows processed
79110 rows processed
79120 rows processed
79130 rows processed
79140 rows processed
79150 rows processed
79160 rows processed
79170 rows processed
79180 rows processed
79190 rows pr

,Mag20,Mag25,Mag30,Mag35,Mag40,Mag45,Mag50,Mag55,Mag60,Mag65,Mag70,Mag75,Mag80,Mag85,Mag90
0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,7.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,6.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,16.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,19.0,9.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,18.0,9.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
df.to_csv('features.csv')

In [117]:
index = np.arange(0,len(df))
df.insert(0,'index',index)


In [119]:
df.set_index('index',inplace = True, drop=True)
df.head()

,Mag20,Mag25,Mag30,Mag35,Mag40,Mag45,Mag50,Mag55,Mag60,Mag65,Mag70,Mag75,Mag80,Mag85,Mag90
index,,,,,,,,,,,,,,,
0,0.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,7.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,6.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,16.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,6.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [121]:
Main_df.insert(0,'index',index)
Main_df.set_index('index',inplace = True, drop=True)
Main_df.head()



,time,latitude,longitude,depth,mag,ClosestCity,category,lastMonth,POR
index,,,,,,,,,
0,1975-12-31 02:53:00.00,34.279000,-116.337000,2.330,2.70,Twenty Nine Palms,2.5,1975-12-01 02:53:00.00,0.014476
1,1975-12-30 23:37:00.00,36.905667,-121.637167,-0.155,2.67,Salinas,2.5,1975-11-30 23:37:00.00,0.020145
2,1975-12-30 23:04:00.00,36.838667,-121.569833,4.774,2.67,Salinas,2.5,1975-11-30 23:04:00.00,0.020145
3,1975-12-30 18:10:00.00,35.781000,-121.421000,11.895,3.15,King City NV,3.0,1975-11-30 18:10:00.00,0.060606
4,1975-12-29 15:07:00.00,36.816333,-121.138833,6.478,3.50,Salinas,3.5,1975-11-29 15:07:00.00,0.171233


In [122]:
combined_df = pd.merge(Main_df, df, on = 'index')
combined_df.head()

,time,latitude,longitude,depth,mag,ClosestCity,category,lastMonth,POR,Mag20,...,Mag45,Mag50,Mag55,Mag60,Mag65,Mag70,Mag75,Mag80,Mag85,Mag90
index,,,,,,,,,,,,,,,,,,,,,
0,1975-12-31 02:53:00.00,34.279000,-116.337000,2.330,2.70,Twenty Nine Palms,2.5,1975-12-01 02:53:00.00,0.014476,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1975-12-30 23:37:00.00,36.905667,-121.637167,-0.155,2.67,Salinas,2.5,1975-11-30 23:37:00.00,0.020145,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1975-12-30 23:04:00.00,36.838667,-121.569833,4.774,2.67,Salinas,2.5,1975-11-30 23:04:00.00,0.020145,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1975-12-30 18:10:00.00,35.781000,-121.421000,11.895,3.15,King City NV,3.0,1975-11-30 18:10:00.00,0.060606,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1975-12-29 15:07:00.00,36.816333,-121.138833,6.478,3.50,Salinas,3.5,1975-11-29 15:07:00.00,0.171233,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
combined_df.to_csv('all features.csv')